In [33]:
import os 
os.chdir('/Users/mattmyers/Desktop/School/tj_poisson_project')
os.getcwd()

'/Users/mattmyers/Desktop/School/tj_poisson_project'

# How I Order Chips for Trader Joe's Using Python, Poisson Distributions, and Power BI. 

Thanks for checking out my project! I work at Trader Joe's and am responsible for ordering inventory for the chip section. This is a fun probability challenge as each day I have to gauge how much product to order in order to meet customer demand, without ordering too much so that our backstock room overflows with blue corn tortilla chips. 

At work, I have access to the average sales per product, which gives me a  sense for what the rough middle of a probability distribution is. The problem is we sell X amount on average but we could sell more than that or less than that on any given day. I knew I could get a more nuanced view using a Poisson Distribution which gives a probability distribution based on an independent event's average rate like an average sale at a retail store. I'll dive into the nitty gritty below, but the Poisson Distribution allows me to see where the likely edge of what possible sales are, so that I can maximize the probability we have enough product to meet demand, without ordering more inventory than is ever likely to be sold. 

The goal in this notebook is to make a lookup table that creates all the different permutations of how many cases I can order, the resulting probability we meet customer demand, and the expected backstock left over. 



## Import Packages
To start let's import the packages we will need. Primarily we will be using the poisson function from the scipy.stats package. 

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import poisson 

## Using Poisson.CDF To Calcualte Probability We Meet Demand

We'll be using two Poisson methods in this notebook, starting with `poisson.cdf`.

`Cdf` stands for **cumulative distribution function**. Given an expected average rate of events (λ, or `mu`), `poisson.cdf(k, mu)` returns the probability that **k or fewer events** will occur — that is, the cumulative probability from 0 through k.

In our ordering scenario, imagine we sell an average of 1.5 cases of chips per day. If we plan to order 2 cases, we can use `poisson.cdf` to calculate the probability that **demand will be 0, 1, or 2 cases** — in other words, the probability that our order **covers the day's demand**.

So, we define `mu = 1.5` and `k = 2`, and compute:

In [25]:
#Average sales and number of cases we plan to order
mu = 1.5  # Expected sales (λ)
k = 2     # number of cases we plan to order (k)

#Probability of selling ≤ 2 cases
prob = poisson.cdf(k, mu)

print(f"Probability of selling less than 2 cases given {mu} cases in average sales: {prob:.2%}.")

Probability of selling less than 2 cases given 1.5 cases in average sales: 80.88%.


Now let's use the same `poisson.cdf` function with a `for` loop to generate a table of probabilities for different shipment sizes.

For example, if a product has **average daily sales of 1.5 cases**, we want to know the probability that ordering 0, 1, 2, ..., up to 5 cases will be **enough to meet demand**. In other words, we’re asking:

> “What’s the probability that demand will be less than or equal to the number of cases we order?”

This gives us a **menu of coverage probabilities** across possible order sizes.

In [26]:
def demand_probability_table(average_sales):
    """
    This function accepts average sales and uses a poisson.cdf method to return a table
    of cases vs the probability i cases will meet demand.
    """
    data=[]
    for i in range (0,6):
        prob = poisson.cdf(i, average_sales)
        data.append({
            "Cases on Hand": i,
            "Probability We Meet Demand": round(prob*100,2)
        })
    
    return pd.DataFrame(data)

demand_probability_table(1.5)

,Cases on Hand,Probability We Meet Demand
0,0,22.31
1,1,55.78
2,2,80.88
3,3,93.44
4,4,98.14
5,5,99.55


So for this hypothetical product we might want to order 3 or 4 cases, which would cover 93.44% or 98.14% of the distribution respectively. 

Let's expand this idea to make a look up table for a range of average sales, a range of cases we might select from 0-20, and add another element-- expected leftover cases. Given that we know our average sales, we can calculate how many cases we will likely have left over. This will simply be the average sales subtracted from the cases we select to order. 

To accomplish this we'll use 2 for loops. The first gives us the average sales range. The itterates over 0 to 20 to calculate our probability demand is met using the poisson.cdf method. Additionally we get the expected leftover cases by subtracting the selected average sales from the selected number of cases ordered. Here is our function: 

### Full Poisson-Based Order Planning Table

Now let’s expand this idea to build a more comprehensive **lookup table**. We'll include:
- A **range of average sales values** (λ)
- A **range of possible order quantities** (0–20 cases)
- The **probability that demand will be met** for each order size
- The **expected number of leftover cases**

This tool can help us dynamically explore how different combinations of average demand and inventory levels affect fulfillment and waste.

#### Key Calculations:
- **Probability Demand is Met**: Using `poisson.cdf()`, we calculate the probability that daily demand will be **less than or equal to** the number of cases ordered.
- **Expected Sales**: We use the sum of `min(k, i) * poisson.pmf(k, λ)` across a range of possible demand values to estimate how much we will realistically sell when `i` cases are on hand.
- **Expected Leftover**: We subtract the expected sales from the number of cases on hand:  

This gives us a table that supports **order optimization** — balancing the risk of stockouts with the cost of overstocking.

In [27]:
def full_order_table(average_sales_range, max_cases = 20):
    data = []

    for average_sales in average_sales_range: 
        for i in range(0,max_cases+1):
            prob_demand = poisson.cdf(i,average_sales)
            expected_sales = sum(min(k,i)*poisson.pmf(k,average_sales) for k in range(0,i+10))
            expected_leftover = round(i-expected_sales,2)

            data.append({
                "Average Sales": round(average_sales, 2),
                "Cases On Hand": i,
                "Probability Demand Met": round(prob_demand*100,2),
                "Expected_Leftover": expected_leftover
            })

    return pd.DataFrame(data)

### Generating the Full Lookup Table for Average Sales from 0.5 to 30 Cases

To populate our full lookup table, we’ll generate a range of **average daily sales values** using a list comprehension. In my store, products typically average between **0.5 and 20 cases per day**, but we’ll extend the range up to **30 cases** to ensure we cover high-volume scenarios.

We'll increment by **0.1 case** at a time for fine-grained resolution. Here's how we create that input range and pass it into our `full_order_table` function:

In [28]:
df = full_order_table(average_sales_range = [x/10 for x in range(5,301)])

df.head(20)

,Average Sales,Cases On Hand,Probability Demand Met,Expected_Leftover
0,0.5,0,60.65,0.00
1,0.5,1,90.98,0.61
2,0.5,2,98.56,1.52
3,0.5,3,99.82,2.50
4,0.5,4,99.98,3.50
5,0.5,5,100.00,4.50
6,0.5,6,100.00,5.50
7,0.5,7,100.00,6.50
8,0.5,8,100.00,7.50
9,0.5,9,100.00,8.50


Now let's save this dataframe as a .CSV so we can import it later into Power BI. 

In [29]:
df.to_csv("inventory_tradeoff_table.csv", index=False)

## Creating Probability Distributions Using Poisson.pmf
There's one more layer of probability nuance I'd like to bring into my Power BI report. While the expected number of leftover cases is helpful, it might also be useful to visualize the full probability distribution of possible leftover case counts. In other words, what is the exact probability of having 0, 1, 2, 3, etc., cases left over?

To calculate this, I'll use the poisson.pmf method. “PMF” stands for probability mass function, and it returns the exact probability of a specific number of events occurring, given an expected average rate (λ). In this case, it tells us the probability of ending up with each possible number of leftover cases.

I’ll use this later to visualize the distribution of leftover case probabilities for a given shipment.

This function uses nested for loops to generate a detailed probability distribution table showing the likelihood of having a specific number of leftover cases after a shipment.

For each combination of:

    Average sales rate (λ)

    Cases on hand (stock ordered)

    Leftover cases (ranging from 0 to full stock),

…the function calculates the probability of that leftover scenario by subtracting the leftover from the stock on hand to estimate actual demand, and then applying poisson.pmf() to calculate the probability of that demand occurring.

The result is a table with all possible combinations of:

    Average sales

    Cases on hand

    Leftover cases

    The probability (%) of that outcome

This table can be used to build visuals that show the full distribution of potential inventory outcomes for different average sales levels — not just the expected leftover.

In [30]:
def leftover_distribution_table(average_sales_range, max_cases=20):
    data = []

    for avg_sales in average_sales_range:
        for stock_on_hand in range(0, max_cases + 1):
            for i in range(0, stock_on_hand + 1):  # Valid leftover range
                demand = stock_on_hand - i
                prob = poisson.pmf(demand, avg_sales)

                data.append({
                    "Average Sales": round(avg_sales, 2),
                    "Cases on Hand": stock_on_hand,
                    "Leftover Cases": i,
                    "Probability (%)": round(prob * 100, 2)
                })

    return pd.DataFrame(data)

Now that we have our function let's fill a dataframe with average sales from 0.5 to 30 cases in increments of 0.1 using the same list comprehension as before. 

In [31]:
avg_sales_range = [x / 10 for x in range(5, 3001)]  # 0.5 to 30
df2 = leftover_distribution_table(avg_sales_range)
df2.head(10)

,Average Sales,Cases on Hand,Leftover Cases,Probability (%)
0,0.5,0,0,60.65
1,0.5,1,0,30.33
2,0.5,1,1,60.65
3,0.5,2,0,7.58
4,0.5,2,1,30.33
5,0.5,2,2,60.65
6,0.5,3,0,1.26
7,0.5,3,1,7.58
8,0.5,3,2,30.33
9,0.5,3,3,60.65


In [19]:
df2.shape

(692076, 4)

We can see this is a very large number of rows, but being under a million rows should still be within the range of what Power BI can handle. Let's create a .CSV that we can use in Power BI. 

In [32]:
df2.to_csv("leftover_distribution_table.csv", index=False)

## Conclusion

In this project, I used the poisson.cdf and poisson.pmf methods to build two complementary lookup tables designed to support smarter inventory planning. The first table calculates the probability that a given number of cases on hand will meet daily demand, using the cumulative distribution function to estimate service levels across a range of average sales and order quantities. The second table uses the probability mass function to calculate the likelihood of each possible leftover amount, along with the expected number of leftover cases. Together, these tables form the backbone of a dynamic Power BI model that allows users to input average sales and visually explore the trade-off between stocking too little and too much, all grounded in a probabilistic understanding of real-world demand variation.